# tsam - Optimal combination of segments and periods for building supply systems 
Date: 29.05.2022

Author: Leander Kotzur

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import tsam
import tsam.hyperparametertuning as tune
from tsam.timeseriesaggregation import TimeSeriesAggregation

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={
        "T": "Temperature [°C]",
        "Load": "Load [kW]",
        "Wind": "Wind [m/s]",
        "GHI": "Solar [W/m²]",
    }
)

In [ ]:
raw.drop(
    columns=[
        "Wind [m/s]",
    ],
    inplace=True,
)

Use tsam's built-in heatmap plotting for visual comparison of the time series

Plot an example series - in this case the temperature

In [ ]:
# Original data heatmaps
tsam.plot_heatmaps(raw, period_hours=24, title="Original Data")

### Tune a hierarchical aggregation with segments in combination with duration representation

In [ ]:
tunedAggregations = tune.HyperTunedAggregations(
    TimeSeriesAggregation(
        raw,
        hoursPerPeriod=24,
        clusterMethod="hierarchical",
        representationMethod="durationRepresentation",
        distributionPeriodWise=False,
        rescaleClusterPeriods=False,
        segmentation=True,
    )
)

And determine the pareto optimal aggregation up to 200 total time steps. This may take some time...

In [ ]:
tunedAggregations.identifyParetoOptimalAggregation(untilTotalTimeSteps=100)

And show the results for the last aggregation

In [ ]:
predictedPeriods = tunedAggregations.aggregationHistory[-1].predictOriginalData()

In [ ]:
# Reconstructed data heatmaps from last tuned aggregation
tsam.plot_heatmaps(
    predictedPeriods,
    reference_data=raw,
    period_hours=24,
    title="Reconstructed Data (Tuned)",
)

In [ ]:
tunedAggregations._segmentHistory[-1]

In [ ]:
tunedAggregations._periodHistory[-1]

In [ ]:
aggregation = TimeSeriesAggregation(
    raw,
    hoursPerPeriod=24,
    noSegments=8,
    noTypicalPeriods=14,
    clusterMethod="hierarchical",
    rescaleClusterPeriods=False,
    segmentation=True,
    representationMethod="distributionAndMinMaxRepresentation",
    distributionPeriodWise=False,
)

In [ ]:
# Reconstructed data heatmaps with 8 segments and 14 periods
tsam.plot_heatmaps(
    aggregation.predictOriginalData(),
    reference_data=raw,
    period_hours=24,
    title="Reconstructed Data (8 segments, 14 periods)",
)